In [1]:
import sys
sys.path.append("/Users/nrahaman/Python/info-bazaar")
sys.path.append("/Users/nrahaman/Python/speedrun")

In [72]:
import pandas as pd
import seaborn as sns
from pathlib import Path
from itertools import product
from tqdm import tqdm
import random
import numpy as np

from bazaar.py_utils import load_dict, dump_dict

In [114]:
user = input("who u? nasim / martin")

who u? nasim / martin nasim


In [115]:
if user == "nasim": 
    noinspect_path = Path("/Users/nrahaman/Downloads/likert_debate_eval_gpt-3.5-turbo_fupnoinspect-STAR-RemoteLlama-2-70b-chat-hf-STAR.csv")
    normal_path = Path("/Users/nrahaman/Downloads/likert_debate_eval_gpt-3.5-turbo_fup-v3-STAR-RemoteLlama-2-70b-chat-hf-STAR.csv")
else: 
    raise NotImplementedError("plz fill")

In [17]:
noinspect_df = pd.read_csv(noinspect_path)
normal_df = pd.read_csv(normal_path)

In [18]:
def get_unique_qs(df): 
    return df["question"].unique().tolist()


def keep_at_budget(df, budget): 
    return df[df["budget"].round() == budget]


def keep_at_question(df, question): 
    return df[df["question"] == question]

In [19]:
shared_qs = list(set(get_unique_qs(noinspect_df)).intersection(set(get_unique_qs(normal_df))))

In [116]:
if user == "nasim": 
    pre_humaneval_path = "./assets/verdicts_pre-humaneval.json"
else: 
    raise NotImplementedError("plz fill")
    
shuffled_verdicts = load_dict(pre_humaneval_path)

In [117]:
for v in shuffled_verdicts: 
    question = v["question"]
    budget_a, budget_b = v["budget_pair"]
    budget_a_answer = keep_at_budget(keep_at_question(normal_df, question), budget_a).iloc[0].answer
    budget_b_answer = keep_at_budget(keep_at_question(normal_df, question), budget_b).iloc[0].answer
    flip = random.randint(0, 1) == 1
    
    if f"human_verdict_{user}" in v: 
        continue
        
    has_been_looked_at_by_human = len([k for k in v if k.startswith("human_verdict_")]) > 0
    
    if not has_been_looked_at_by_human: 
        continue
    
    print("Question:", question)
    print("-" * 50)
    if not flip: 
        print("Answer A:", budget_a_answer)
        print("-" * 50)
        print("Answer B:", budget_b_answer)
        print("-" * 50)
    else: 
        print("Answer A:", budget_b_answer)
        print("-" * 50)
        print("Answer B:", budget_a_answer)
        print("-" * 50)
    better = input("Which one better?")
    if better in ["A", "B"]: 
        if flip: 
            better = {"A": "B", "B": "A"}[better]
        v[f"human_verdict_{user}"] = better
    elif better == "q": 
        break
    elif better == "c": 
        continue
    else: 
        raise ValueError
    print("-" * 50)
    print("-" * 50)
    print()
    
    

In [108]:
human_verdicts = [v for v in shuffled_verdicts if f"human_verdict_{user}" in v]

In [110]:
matches = [(1 if v[f"human_verdict_{user}"] == v["verdict"] else 0) for v in human_verdicts]

In [111]:
np.mean(matches)

0.7

In [112]:
len(matches)

50

In [113]:
_ = dump_dict(shuffled_verdicts, "./assets/verdicts_pre-humaneval.json")